In [102]:
import argparse
from os.path import dirname
import torch
import torchvision
import os
import numpy as np
import tqdm

from utils.models import PreprocessLayer
from utils.models import Classifier
from torch.utils.tensorboard import SummaryWriter
from utils.loader import Loader
from utils.loss import cross_entropy_loss_and_accuracy
from utils.dataset import NCaltech101
from torch.utils.data.dataloader import default_collate


In [103]:
torch.manual_seed(777)
np.random.seed(777)

In [104]:
validation_dataset="/ws/data/N-Caltech101/validation/"
training_dataset="/ws/data/N-Caltech101/training/"
log_dir="/ws/external/log_1.4_b4/temp"
device="cuda:0"
num_workers=4
pin_memory=True
batch_size=4
num_epochs=2
save_every_n_epochs=2
checkpoint = "/ws/external/log_1.4_b4/model_best.pth" # model_best.pth checkpoint_13625_0.5990.pth
    
    
assert os.path.isdir(dirname(log_dir)), f"Log directory root {dirname(log_dir)} not found."
assert os.path.isdir(validation_dataset), f"Validation dataset directory {validation_dataset} not found."
assert os.path.isdir(training_dataset), f"Training dataset directory {training_dataset} not found."

print(f"----------------------------\n"
      f"Starting training with \n"
      f"num_epochs: {num_epochs}\n"
      f"batch_size: {batch_size}\n"
      f"device: {device}\n"
      f"log_dir: {log_dir}\n"
      f"training_dataset: {training_dataset}\n"
      f"validation_dataset: {validation_dataset}\n"
      f"----------------------------")




----------------------------
Starting training with 
num_epochs: 2
batch_size: 4
device: cuda:0
log_dir: /ws/external/log_1.4_b4/temp
training_dataset: /ws/data/N-Caltech101/training/
validation_dataset: /ws/data/N-Caltech101/validation/
----------------------------


In [105]:
def percentile(t, q):
    B, C, H, W = t.shape
    k = 1 + round(.01 * float(q) * (C * H * W - 1))
    result = t.view(B, -1).kthvalue(k).values
    return result[:,None,None,None]

def create_image(representation):
    B, C, H, W = representation.shape
    representation = representation.view(B, 3, C // 3, H, W).sum(2)

    # do robust min max norm
    representation = representation.detach().cpu()
    robust_max_vals = percentile(representation, 99)
    robust_min_vals = percentile(representation, 1)

    representation = (representation - robust_min_vals)/(robust_max_vals - robust_min_vals)
    representation = torch.clamp(255*representation, 0, 255).byte()

    representation = torchvision.utils.make_grid(representation)

    return representation
    

In [106]:
class Loader:
    def __init__(self, dataset, batch_size=2, num_workers=2, pin_memory=True, device="cuda:0"):
        self.device = device
        split_indices = list(range(len(dataset)))
        sampler = torch.utils.data.sampler.SubsetRandomSampler(split_indices)
        self.loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=sampler,
                                             num_workers=num_workers, pin_memory=pin_memory,
                                             collate_fn=collate_events)

    def __iter__(self):
        for data in self.loader:
            data = [d.to(self.device) for d in data]
            yield data

    def __len__(self):
        return len(self.loader)

def collate_events(data):
    labels = []
    events = []
    for i, d in enumerate(data):
        labels.append(d[1])
        ev = np.concatenate([d[0], i*np.ones((len(d[0]),1), dtype=np.float32)],1)
        events.append(ev)
    events = torch.from_numpy(np.concatenate(events,0))
    labels = default_collate(labels)
    return events, labels

In [107]:
# datasets, add augmentation to training set
training_dataset = NCaltech101(training_dataset, augmentation=True)
validation_dataset = NCaltech101(validation_dataset)

# construct loader, handles data streaming to gpu
training_loader = Loader(training_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, device="cuda:0")
validation_loader = Loader(validation_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, device="cuda:0")

In [108]:
# model, and put to device
preprocess = PreprocessLayer()
model = Classifier(pretrained=False)
ckpt = torch.load(checkpoint)
model.load_state_dict(ckpt["state_dict"])
model = model.to(device)

# # optimizer and lr scheduler
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.5)

writer = SummaryWriter(log_dir)

iteration = 0
min_validation_loss = 1000

In [109]:
_, (events, labels) = next(enumerate(validation_loader))

In [110]:
np.shape(events)

torch.Size([624273, 5])

In [111]:
labels

tensor([67, 21, 63, 41], device='cuda:0')

In [112]:
# optimizer.zero_grad()
t = preprocess(events)
pred_labels, representation = model(events, t)
loss, accuracy = cross_entropy_loss_and_accuracy(pred_labels, labels)
# loss.backward()
# optimizer.step()

In [113]:
pred_labels.argmax(dim=1)

tensor([97, 21, 63, 12], device='cuda:0')

In [114]:
np.shape(pred_labels)

torch.Size([4, 101])

In [115]:
labels

tensor([67, 21, 63, 41], device='cuda:0')

In [116]:
# representation_vizualization = create_image(representation)
# writer.add_image("training/representation", representation_vizualization, iteration)

In [117]:
IMAGE_SCALE=255

class PGDAttacker():
    def __init__(self, model, num_iter, epsilon, step_size, kernel_size=15, 
                 prob_start_from_clean=0.0, translation=False, num_classes=101, device='cuda:0'):
        self.model = model
        self.model.eval()
        step_size = max(step_size, epsilon / num_iter)
        self.num_iter = num_iter
        self.epsilon = epsilon * IMAGE_SCALE
        self.step_size = step_size*IMAGE_SCALE
        self.prob_start_from_clean = prob_start_from_clean
        self.device = device
        self.translation = translation
        self.num_classes = num_classes


        if translation:
            # this is equivalent to deepth wise convolution
            # details can be found in the docs of Conv2d.
            # "When groups == in_channels and out_channels == K * in_channels, where K is a positive integer, this operation is also termed in literature as depthwise convolution."
            self.conv = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=kernel_size, stride=(kernel_size-1)//2, bias=False, groups=3).to(self.device)
            self.gkernel = get_kernel(kernel_size, nsig=3, device=self.device).to(self.device)
            self.conv.weight = self.gkernel

    def _create_random_target(self, label):
        label_offset = torch.randint_like(label, low=0, high=self.num_classes)
        return (label + label_offset) % self.num_classes

    def attack(self, events, label, t, original=False, mode='pgd'):
        if mode == 'pgd':
            return self.pgd_attack(events, label, t, original=False)


    def set_model(self, model):
        self.model = model

    def pgd_attack(self, events, label, t, original=False):
        """
        aux_images, _ = self.attacker.attack(x, labels, self._forward_impl)
        """
        if original:
            target_label = label    # untargeted
        else:
            target_label = self._create_random_target(label)    # targeted
#         lower_bound = torch.clamp(image_clean - self.epsilon, min=-1., max=1.)
#         upper_bound = torch.clamp(image_clean + self.epsilon, min=-1., max=1.)

        ori_images = t.clone().detach()

        init_start = torch.empty_like(t).uniform_(-self.epsilon, self.epsilon)
        
#         start_from_noise_index = (torch.randn([])>self.prob_start_from_clean).float() 
#         start_adv = t + start_from_noise_index * init_start
        
        start_adv = t

        adv = start_adv
        adv.requires_grad = True
        for i in range(self.num_iter):
            
            logits, representation = self.model(events, adv)
            
            losses = torch.nn.functional.cross_entropy(logits, target_label)
            g = torch.autograd.grad(losses, adv, 
                                    retain_graph=False, create_graph=False)[0]
            if self.translation:
                g = self.conv(g)
            # Linf step
            if original:
                adv = adv + torch.sign(g) * self.step_size  # untargeted
            else:
                adv = adv - torch.sign(g) * self.step_size  # targeted
#             # Linf project
#             adv = torch.where(adv > lower_bound, adv, lower_bound).detach()
#             adv = torch.where(adv < upper_bound, adv, upper_bound).detach()
        
        return adv, target_label, t

In [118]:
num_iter = 2
epsilon = 1
step_size = 0.5
attacker = PGDAttacker(model, num_iter, epsilon, step_size, kernel_size=15, 
            prob_start_from_clean=0.0, translation=False, num_classes=101, device='cuda:0')
adv, target_label, t = attacker.attack(events, labels, t, original=False, mode='pgd')

In [121]:
np.shape(adv)

torch.Size([624273])

In [122]:
np.shape(events)

torch.Size([624273, 5])

In [123]:
init_start

tensor([ -79.8654, -132.1838, -150.6050,  ...,  -94.5678,  -95.1334,
         -81.0742], device='cuda:0')

In [124]:
np.shape(init_start)

torch.Size([624273])

In [125]:
np.shape(logits)

torch.Size([4, 101])

In [126]:
np.shape(t)

torch.Size([624273])

In [127]:
target_label

tensor([23, 89, 30, 79], device='cuda:0')

In [128]:
labels

tensor([67, 21, 63, 41], device='cuda:0')

In [5]:

for i in range(num_epochs):
    sum_accuracy = 0
    sum_loss = 0
    model = model.eval()

    print(f"Validation step [{i:3d}/{num_epochs:3d}]")
    for events, labels in tqdm.tqdm(validation_loader):

        with torch.no_grad():
            pred_labels, representation = model(events)
            loss, accuracy = cross_entropy_loss_and_accuracy(pred_labels, labels)

        sum_accuracy += accuracy
        sum_loss += loss

    validation_loss = sum_loss.item() / len(validation_loader)
    validation_accuracy = sum_accuracy.item() / len(validation_loader)

    writer.add_scalar("validation/accuracy", validation_accuracy, iteration)
    writer.add_scalar("validation/loss", validation_loss, iteration)

    # visualize representation
    representation_vizualization = create_image(representation)
    writer.add_image("validation/representation", representation_vizualization, iteration)

    print(f"Validation Loss {validation_loss:.4f}  Accuracy {validation_accuracy:.4f}")

    if validation_loss < min_validation_loss:
        min_validation_loss = validation_loss
        state_dict = model.state_dict()

        torch.save({
            "state_dict": state_dict,
            "min_val_loss": min_validation_loss,
            "iteration": iteration
        }, "log/model_best.pth")
        print("New best at ", validation_loss)

    if i % save_every_n_epochs == 0:
        state_dict = model.state_dict()
        torch.save({
            "state_dict": state_dict,
            "min_val_loss": min_validation_loss,
            "iteration": iteration
        }, "log/checkpoint_%05d_%.4f.pth" % (iteration, min_validation_loss))

    sum_accuracy = 0
    sum_loss = 0

    model = model.train()
    print(f"Training step [{i:3d}/{num_epochs:3d}]")
    for events, labels in tqdm.tqdm(training_loader):
        optimizer.zero_grad()

        pred_labels, representation = model(events)
        loss, accuracy = cross_entropy_loss_and_accuracy(pred_labels, labels)

        loss.backward()

        optimizer.step()

        sum_accuracy += accuracy
        sum_loss += loss

        iteration += 1

    if i % 10 == 9:
        lr_scheduler.step()

    training_loss = sum_loss.item() / len(training_loader)
    training_accuracy = sum_accuracy.item() / len(training_loader)
    print(f"Training Iteration {iteration:5d}  Loss {training_loss:.4f}  Accuracy {training_accuracy:.4f}")

    writer.add_scalar("training/accuracy", training_accuracy, iteration)
    writer.add_scalar("training/loss", training_loss, iteration)

    representation_vizualization = create_image(representation)
    writer.add_image("training/representation", representation_vizualization, iteration)

usage: Train classifier using a learnt quantization layer. [-h]
                                                           --validation_dataset
                                                           VALIDATION_DATASET
                                                           --training_dataset
                                                           TRAINING_DATASET
                                                           --log_dir LOG_DIR
                                                           [--device DEVICE]
                                                           [--num_workers NUM_WORKERS]
                                                           [--pin_memory PIN_MEMORY]
                                                           [--batch_size BATCH_SIZE]
                                                           [--num_epochs NUM_EPOCHS]
                                                           [--save_every_n_epochs SAVE_EVERY_N_EPOCHS]
Train classifier using a 

SystemExit: 2

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
